In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime as dt

from glob import iglob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = filepath

df = pd.concat((pd.read_csv(f) for f in iglob(path, recursive=True)), ignore_index=True)

# Get MMI data
df_mmi = df[df['Type'] == 'meanMotionIntensity']

df_mmi['Start Time'] = df_mmi['Start Time'].apply(pd.to_datetime)

df_mmi['End Time'] = df_mmi['End Time'].apply(pd.to_datetime)

df_mmi['Start_Time_Offset'] = df_mmi['Start Time'] + pd.to_timedelta(df_mmi['Time Zone Offset'], unit='s')
df_mmi['End_Time_Offset'] = df_mmi['End Time'] + pd.to_timedelta(df_mmi['Time Zone Offset'], unit='s')

# get steps data
df_steps = df[df['Type'] == 'steps']
df_steps['User UUID'].unique()

df_steps['Start Time'] = df_steps['Start Time'].apply(pd.to_datetime)

df_steps['End Time'] = df_steps['End Time'].apply(pd.to_datetime)

In [ ]:
df_steps['Start_Time_Offset'] = df_steps['Start Time'] + pd.to_timedelta(df_steps['Time Zone Offset'], unit='s')
df_steps['End_Time_Offset'] = df_steps['End Time'] + pd.to_timedelta(df_steps['Time Zone Offset'], unit='s')

# df['Start_Time_Offset'] = df.loc[:,['Start Time','Time Zone Offset']].sum(axis=1)

df_merge_steps_mi = pd.merge(df_steps, df_mmi, left_on=['User UUID', 'Start_Time_Offset', 'End_Time_Offset'], right_on=['User UUID', 'Start_Time_Offset', 'End_Time_Offset'])
df_merge_steps_mi['User UUID'].unique()

In [ ]:
len(df_merge_steps_mi)

df_merge_steps_mi

df_merge_steps_mi.columns

In [ ]:
# Remove value_y from duplicate subset
df_merge_dup = df_merge_steps_mi.drop_duplicates(subset=['User UUID', 'Start_Time_Offset', 'End_Time_Offset', 'Value_x'], keep='first')
df_merge_dup

sixtyone_epoch = df_merge_dup[df_merge_dup['User UUID']=='u-22Kw0lWnvI27MXMuRggpVA']
sixtyone_epoch = sixtyone_epoch[sixtyone_epoch['Start Time_x'] == '2021-02-28T15:45:00Z']
sixtyone_epoch[['User UUID', 'End Time_x', 'End Time_y', 'Value_y', 'Value_x']]


In [4]:
df_merge_dup['End Time'] = df_merge_dup['End Time_x'].apply(pd.to_datetime)

df_merge_dup['Start_Time_Date_Offset_x'] = df_merge_dup['Start Time Offset'].dt.date
df_merge_dup['End_Time_Date_Offset_x'] = df_merge_dup['End Time Offset'].dt.date

df_merge_dup['month_year'] = df_merge_dup['Start Time Offset'].dt.to_period('M')

# df_merge_dup['Total_Steps_Per_Day'] = df_merge_dup.groupby(['Start_Time_Date_Offset_x', 'User UUID'])['Value_x'].transform(sum())

df_merge_dup['Total_Steps_Per_Day_pre_missing'] = df_merge_dup.groupby(['Start_Time_Date_Offset_x', 'User UUID'])['Value_x'].transform('sum')

# df_merge_steps_mi_dup_z_nz_2['Average_Monthly_Steps_pre_missing'] = df_merge_steps_mi_dup_z_nz_2.groupby([' User UUID', 'month_year'])['Total_Steps_Per_Day'].transform('mean')

df_merge_dup['Mon_Year'] = df_merge_dup['Start_Time_Offset'].dt.strftime('%b-%Y')

df_merge_dup['Month_Year_Name'] = df_merge_dup['Mon_Year']

df_merge_dup.loc[(df_merge_dup['Start_Time_Date_Offset_x'] == dt.date(2020, 3, 15)), 'Month_Year_Name'] = '1-14 Mar-2020'
df_merge_dup.loc[(df_merge_dup['Start_Time_Date_Offset_x'] > dt.date(2020, 3, 15)) & (df_merge_dup['Start_Time_Date_Offset_x'] <= dt.date(2020, 3, 31)), 'Month_Year_Name'] = '15-31 Mar-2020'
df_merge_dup.loc[(df_merge_dup['Start_Time_Date_Offset_x'] == dt.date(2020, 4, 1)), 'Month_Year_Name'] = df_merge_dup.Month_Year_Name

df_merge_dup['Average_Monthly_Steps_Lockdown_pre_missing'] = df_merge_dup.groupby(['User UUID', 'Month_Year_Name'])['Total_Steps_Per_Day_pre_missing'].transform('mean')


In [ ]:
df_zero = df_merge_dup[df_merge_dup['Value_y'] < 1]
df_zero['Value_y'].unique()

df_non_zero = df_merge_dup[df_merge_dup['Value_y'] > 1]
df_non_zero
df_non_zero['User UUID'].unique()

# Calculate missingness
# df_steps_missing = df_merge_steps_mi_dup_non_zero
df_steps_missing = df_non_zero

df_steps_missing['Num_of_days'] = 0
df_steps_missing['total_percent_missing'] = 0
df_steps_missing['percent_missing_per_day'] = 0
df_steps_missing['Epochs_per_day'] = 0



In [ ]:


df_steps_missing['Valid Day'] = 'no'
df_steps_missing.loc[(df_steps_missing['Epochs_per_day'] > 40), 'Valid Day'] = 'yes'

# df_steps_missing_Lockdown_all_years_copy = df_steps_missing

# restrict time
df_all_periods = df_steps_missing[(df_steps_missing['Start_Time_Date_Offset_x'] > dt.date(2019, 3, 1)) & (df_steps_missing['Start_Time_Date_Offset_x'] < dt.date(2021, 6, 30))]

print(df_all_periods['Start_Time_Date_Offset_x'].min())
print(df_all_periods['Start_Time_Date_Offset_x'].max())

df_all_periods.loc[(df_all_periods['Start_Time_Date_Offset_x'] < dt.date(2020, 3, 15)) & (df_all_periods['Start_Time_Date_Offset_x'] > dt.date(2019, 3, 1)), 'closure_status'] = 'Pre Closure'
df_all_periods.loc[(df_all_periods['Start_Time_Date_Offset_x'] < dt.date(2021, 3, 31)) & (df_all_periods['Start_Time_Date_Offset_x'] > dt.date(2020, 3, 15)), 'closure_status'] = 'During Closure'
df_all_periods.loc[(df_all_periods['Start_Time_Date_Offset_x'] > dt.date(2021, 3, 31)), 'closure_status'] = 'Post Closure'

print(df_all_periods.groupby('closure_status')['Start_Time_Date_Offset_x'].min())
print(df_all_periods.groupby('closure_status')['Start_Time_Date_Offset_x'].max())

In [6]:
df_valid_days = df_all_periods[df_all_periods['Valid Day'] == 'yes']

df_valid_days['Num_of_days_valid_epochs'] = df_valid_days.groupby(['User UUID'])['Start_Time_Date_Offset_x'].transform('nunique')

df_valid_days_daycount = df_valid_days[df_valid_days['Num_of_days_valid_epochs'] > 60]

df_valid_days_daycount['Epochs_per_day'].min()
df_valid_days_daycount['Epochs_per_day'].max()

epoch_count = df_all_periods
epoch_count['Num_of_days_valid_epochs'] = epoch_count.groupby(['User UUID'])['Start_Time_Date_Offset_x'].transform('nunique')

epoch_count = df_all_periods.drop_duplicates(subset=['User UUID', 'Num_of_days_valid_epochs'], keep='first')


print(range(len(epoch_count_only)))
print(194/2)

epoch_count_only.max()


In [ ]:
# plt.figure(figsize=(100,40))
plt.figure(figsize=(60,20))
sns.set(font_scale=5)
default_x_ticks = range(len(epoch_count_only))
fig, ax = plt.subplots(figsize=(60,20))
ax.hist(epoch_count_only['Num_of_days_valid_epochs'], bins=61, cumulative=-1, density=False)
ax.set_facecolor('white')

epoch_count_only[epoch_count_only['Num_of_days_valid_epochs'] > 120]['User UUID'].nunique()

epoch_count_only[epoch_count_only['Num_of_days_valid_epochs'] > 61].nunique()

epoch_count_only[epoch_count_only['Num_of_days_valid_epochs'] > 40].nunique()

epoch_count_only[epoch_count_only['Num_of_days_valid_epochs'] > 70].nunique()

epoch_count_only[epoch_count_only['Num_of_days_valid_epochs'] > 80].nunique()

epoch_count_only[epoch_count_only['Num_of_days_valid_epochs'] > 90].nunique()

In [ ]:
# Heatmap
df_epoch_dup = df_all_periods.drop_duplicates(subset=['User UUID', 'Start_Time_Date_Offset_x'], keep='last', inplace=False)
df2 = df_epoch_dup.groupby(['User UUID', 'Epochs_per_day'])['Start_Time_Date_Offset_x'].count()

df3 = df2.reset_index()

df3['Epochs_per_day'] = df3['Epochs_per_day'].replace(np.nan, 0)
df3['Start_Time_Date_Offset_x'] = df3['Start_Time_Date_Offset_x'].replace(np.nan, 0)

df_pivot = df3.pivot_table(values='User UUID', index=['Epochs_per_day'], columns=['Start_Time_Date_Offset_x'], aggfunc=pd.Series.nunique)
df_pivot.replace(np.nan, 0, inplace=True)

sns.set(rc={"figure.figsize": (20, 7)})
sns.set_context("paper", rc={"font.size":90,"axes.titlesize":50,"axes.labelsize":20})
sns.set(rc={"axes.facecolor":"snow", "figure.facecolor":"white"})
ax = sns.heatmap(df_pivot, cmap="rocket_r", mask=(df_pivot==0))
ax.invert_yaxis()

df_valid_days_daycount['User UUID'].nunique()

df_valid_days_daycount['Total_Steps_Per_Day_post_missing'] = df_valid_days_daycount.groupby(['Start_Time_Date_Offset_x', 'User UUID'])['Total_Steps_Per_Day'].transform('sum')

df_valid_days_daycount['Average_Monthly_Steps_Lockdown_post_missing'] = df_valid_days_daycount.groupby(['User UUID', 'Month_Year_Name'])['Total_Steps_Per_Day_post_missing'].transform('mean')

# 

In [ ]:
df_valid_days_daycount_dup = df_valid_days_daycount.drop_duplicates(subset=['User UUID', 'Average_Monthly_Steps_Lockdown_post_missing', 'Month_Year_Name'], keep='first')
df_valid_days_daycount_dup

# Min
df_valid_days_daycount_dup.groupby('closure_status')['Average_Monthly_Steps_Lockdown_post_missing'].min()

# Max
df_valid_days_daycount_dup.groupby('closure_status')['Average_Monthly_Steps_Lockdown_post_missing'].max()

# Mean
df_valid_days_daycount_dup.groupby('closure_status')['Average_Monthly_Steps_Lockdown_post_missing'].mean()

# Std
df_valid_days_daycount_dup.groupby('closure_status')['Average_Monthly_Steps_Lockdown_post_missing'].std()

# Users
df_valid_days_daycount_dup.groupby('closure_status')['User UUID'].nunique()

In [ ]:
df_valid_days_daycount_dup_month = df_valid_days_daycount_dup
df_valid_days_daycount_dup_month['average_steps_per_month_post_missing'] = df_valid_days_daycount_dup_month.groupby(['Month_Year_Name'])['Average_Monthly_Steps_Lockdown_post_missing'].transform('mean')
df_valid_days_daycount_dup_month = df_valid_days_daycount_dup_month.drop_duplicates(subset=['average_steps_per_month_post_missing', 'Month_Year_Name'], keep='first')
df_valid_days_daycount_dup_month

df_valid_days_daycount_dup_month['Month_Year_Name'].unique()

# Min
df_valid_days_daycount_dup_month.groupby('closure_status')['average_steps_per_month_post_missing'].min()

# Max
df_valid_days_daycount_dup_month.groupby('closure_status')['average_steps_per_month_post_missing'].max()

# Mean
df_valid_days_daycount_dup_month.groupby('closure_status')['average_steps_per_month_post_missing'].mean()

# Std
df_valid_days_daycount_dup_month.groupby('closure_status')['average_steps_per_month_post_missing'].std()

# Users
df_valid_days_daycount_dup_month.groupby('closure_status')['User UUID'].nunique()

# Exclude summer for TOD comparison with overall
df_valid_days_daycount_dup_month_non_sum = df_valid_days_daycount_dup_month[(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Jun-2020') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Jul-2020') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Aug-2020') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Jun-2019') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Jul-2019') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Aug-2019') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Jun-2021') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Jul-2021') & 
(df_valid_days_daycount_dup_month['Month_Year_Name'] != 'Aug-2021')]
df_valid_days_daycount_dup_month_non_sum

# Mean
df_valid_days_daycount_dup_month_non_sum.groupby('closure_status')['average_steps_per_month_post_missing'].mean()

In [ ]:
# Figure
# df_steps_missing_Lockdown_all_years_valid_days_daycount_fig4['year_only'] = df_steps_missing_Lockdown_all_years_valid_days_daycount

df_valid_days_daycount_dup_fig = df_valid_days_daycount_dup
df_valid_days_daycount_dup_fig['pre_avg'] = 8809.83

plt.figure(figsize=(60, 20))
sns.set(font_scale=5)
sns.set_style("white")
sns.set_palette("Set1", 8, .75)
ax = sns.lineplot(x='Month_Year_Name', y='Average_Monthly_Steps_Lockdown_post_missing', 
                  data=df_valid_days_daycount_dup_fig, color='darkcyan', 
                  linewidth=15, sort=False)
sns.lineplot(x='Month_Year_Name', y='pre_avg', 
             data=df_valid_days_daycount_dup_fig, color='grey', 
             linestyle='--', linewidth=10, sort=False)

plt.setp(ax.get_xticklabels(), rotation=90)
ax.set_xlabel("Month", fontdict={'fontsize': 60, 'fontweight': 'bold'}, labelpad=-2)
ax.set_ylabel("Average Daily Number of Steps Per Month", fontdict={'fontsize': 55, 'fontweight': 'bold'})
ax.set_title("Average Daily Step Values for Participants (Non zero MMI, after missingness analysis)", fontdict={'fontsize': 60, 'fontweight': 'bold'})
ax.axvline("1-14 Mar-2020", color="firebrick", linestyle="--", lw=10)
ax.legend(loc="upper right", frameon=True, fontsize=50)
plt.setp(ax.get_legend().get_texts(), fontsize='50') # for Legend text
plt.setp(ax.get_legend().get_title(), fontsize='50') # for Legend title

In [ ]:
# Statistical test
steps_pre_closure = df_valid_days_daycount_dup[df_valid_days_daycount_dup['closure_status'] == 'Pre Closure']

steps_pre_closure['Monthly_Avg'] = steps_pre_closure.groupby(['Month_Year_Name'])['Average_Monthly_Steps_Lockdown_post_missing'].transform('mean')

steps_pre_closure['month_num'] = steps_pre_closure['Start_Time_Offset'].dt.month
# steps_pre['month_num'].unique()
steps_pre_closure = steps_pre_closure.sort_values(by=['month_num'])
# steps_pre_closure

steps_pre_closure_col = steps_pre_closure[['Month_Year_Name', 'Monthly_Avg']]
steps_pre_closure_dup = steps_pre_closure_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
# steps_pre_closure_dup

steps_dur_closure = df_valid_days_daycount_dup[df_valid_days_daycount_dup['closure_status'] == 'During Closure']

steps_dur_closure['Monthly_Avg'] = steps_dur_closure.groupby(['Month_Year_Name'])['Average_Monthly_Steps_Lockdown_post_missing'].transform('mean')

steps_dur_closure['month_num'] = steps_dur_closure['Start_Time_Offset'].dt.month
# steps_pre['month_num'].unique()
steps_dur_closure = steps_dur_closure.sort_values(by=['month_num'])
# steps_dur_closure

steps_dur_closure_col = steps_dur_closure[['Month_Year_Name', 'Monthly_Avg']]
steps_dur_closure_dup = steps_dur_closure_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
# steps_dur_closure_dup

In [ ]:
from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(steps_dur_closure_dup['Monthly_Avg'], steps_pre_closure_dup['Monthly_Avg'])
print(U1)
print(p)
print(p * 100)

In [ ]:
# Time of day analysis
df_closure_tod = df_valid_days_daycount

df_closure_tod['Start Time Only Time'] = df_closure_tod['Start_Time_Offset'].dt.strftime("%H:%M:%S")
# all_data_merged_steps[all_data_merged_steps['Start Time Only Time'] > '16:15:00']
# base on data instead
df_closure_tod['Time_Of_Day'] = "Night"
df_closure_tod.loc[df_closure_tod['Start Time Only Time'] < '11:00:00', 'Time_Of_Day'] = 'Morning'
df_closure_tod.loc[(df_closure_tod['Start Time Only Time'] >= '07:00:00') &
                   (df_closure_tod['Start Time Only Time'] < '16:00:00'), 'Time_Of_Day'] = 'Afternoon'
df_closure_tod.loc[(df_closure_tod['Start Time Only Time'] >= '11:00:00') &
                   (df_closure_tod['Start Time Only Time'] < '20:00:00'), 'Time_Of_Day'] = 'Evening'

# aft = df_steps_missing_all_dates_copy_fig3[df_steps_missing_all_dates_copy_fig3['Time_Of_Day'] == 'Evening']
# aft['Start Time Only Time'].unique()

df_closure_tod['wkday_Num'] = df_closure_tod['Start_Time_Offset'].dt.weekday
df_closure_tod['wkday_Num'].unique()

df_closure_tod_wkday = df_closure_tod[df_closure_tod['wkday_Num'].isin([0,1,2,3,4])]
df_closure_tod_wkday['wkday_Num'].unique()

df_closure_tod_wkday[['wkday_Num', 'Start_Time_Offset']]

In [ ]:

df_closure_tod_wkday_school = df_closure_tod_wkday[df_closure_tod_wkday['Time_Of_Day'].isin(['Morning', 'Afternoon'])]
df_closure_tod_wkday_school_non_sum = df_closure_tod_wkday_school[~df_closure_tod_wkday_school['Mon_Year'].isin(['Jun-2020', 'Jul-2020', 'Aug-2020', 'Jun-2019', 'Jul-2019', 'Aug-2019', 'Jun-2021', 'Jul-2021', 'Aug-2021'])]
df_closure_tod_wkday_school_non_sum['Daily_Sum_TOD'] = df_closure_tod_wkday_school_non_sum.groupby(['Start_Time_Date_Offset_x', 'User_UUID'])['Steps'].transform('sum')
df_closure_tod_wkday_school_non_sum['mean_monthly_steps_school'] = df_closure_tod_wkday_school_non_sum.groupby(['User_UUID', 'Month_Year_Name'])['Daily_Sum_TOD'].transform('mean')

df_closure_tod_wkday_school_non_sum_dup = df_closure_tod_wkday_school_non_sum.drop_duplicates(subset=['User_UUID', 'mean_monthly_steps_school'])
df_closure_tod_wkday_school_non_sum_dup

In [ ]:
# Max
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['mean_monthly_steps_school'].max()

# Mean
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['mean_monthly_steps_school'].mean()

# Std
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['mean_monthly_steps_school'].std()

# Users
df_closure_tod_wkday_school_non_sum_dup.groupby('closure_status')['User_UUID'].nunique()

In [ ]:
df_closure_tod_fig = df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status'] != 'Post Closure']
plt.figure(figsize=(100, 40))
sns.set_style("white")
palette = {
    "Pre Closure": "mediumpurple",
    "During Closure": "lightcoral"
#     "#2020": "salmon"
}

ax = sns.boxplot(x="Month_Year_Name", y="mean_monthly_steps_school", palette=palette, hue="closure_status",
                 data=df_closure_tod_fig, linewidth=14,
                 showfliers=False, dodge=False, width=0.5,
                 showmeans=True)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
# ax.set_title('Average Step Values for Participants Before and After Lockdown For School Times')
ax.set_xlabel('Month', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_ylabel('Number of Steps', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_title('Average Daily Step Count Values for Participants Before and After Lockdown During School Times',
             fontdict={'fontsize': 90, 'fontweight': 'bold'}, y=1.05)

plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0, frameon=True, edgecolor='black',
           title="Pandemic Status", framealpha=0.5)

In [ ]:
df_closure_tod_fig = df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status'] != 'Post Closure']
fig_2_df_closure_tod = df_closure_tod_fig
# Following lines are not completely visible in the image, but here's the visible part:
fig_2_df_closure_tod['Month Year Name_Closure'] = fig_2_df_closure_tod['Month Year Name'].str[0:3]
fig_2_df_closure_tod.loc[(fig_2_df_closure_tod['Month Year Name_Closure'] == 'Mar'), 'Month Year Name_Closure'] = 'Mar-2020'
fig_2_df_closure_tod.loc[(fig_2_df_closure_tod['Month Year Name_Closure'] == 'Mar'), 'Month Year Name_Closure'] = 'Mar-'
fig_2_df_closure_tod['month only'] = pd.Categorical(fig_2_df_closure_tod['Month Year Name_Closure'],
                                                    categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Sep', 'Oct', 'Nov', 'Dec'], ordered=True)

In [ ]:
# Commented out line
# df_closure_tod_fig=df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status']=='Post Closure']

plt.figure(figsize=(100, 40))
sns.set_style("white")
palette = {"Pre Closure": "mediumpurple",
           "During Closure": "lightcoral",
           "#2020": "salmon"}

ax = sns.boxplot(x="month_only", y="mean_monthly_steps_school", palette=palette, hue="closure_status",
                 data=fig_2_df_closure_tod, linewidth=14, showfliers=False, dodge=False, width=0.5, showmeans=True)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_xlabel('Month', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_ylabel('Number of Steps', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_title('Average Daily Step Count Values for Participants Before and After Lockdown During School Times', 
             fontdict={'fontsize': 90, 'fontweight': 'bold'}, y=1.05)

plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0, frameon=True, edgecolor='black',
           title="Pandemic Status", framealpha=0.5)

In [ ]:
# New one value per month
df_closure_tod_wkday_school_non_sum_dup_month = df_closure_tod_wkday_school_non_sum_dup
df_closure_tod_wkday_school_non_sum_dup_month['mean_monthly_steps_school_per_month'] = df_closure_tod_wkday_school_non_sum_dup_month.groupby(['Month_Year_Name'])['mean_monthly_steps_school'].transform('mean')

df_closure_tod_wkday_school_non_sum_dup_month_dup = df_closure_tod_wkday_school_non_sum_dup_month.drop_duplicates(subset=['mean_monthly_steps_school_per_month', 'Month_Year_Name'], keep='first')

# Min
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_per_month'].min()

# Max
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_per_month'].max()

# Mean
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_per_month'].mean()

# Std
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_per_month'].std()

# Users
df_closure_tod_wkday_school_non_sum_dup_month_dup.groupby('closure_status')['User_UUID'].nunique()

In [ ]:
# Statistical test
steps_pre_closure_wkday_school_non_sum = df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status'] == 'Pre Closure']
steps_pre_closure_wkday_school_non_sum['Monthly_Avg'] = steps_pre_closure_wkday_school_non_sum.groupby(['Month_Year_Name'])['mean_monthly_steps_school'].transform('mean')
steps_pre_closure_wkday_school_non_sum['month_num'] = steps_pre_closure_wkday_school_non_sum['Start_Time_Offset'].dt.month
steps_pre_closure_wkday_school_non_sum = steps_pre_closure_wkday_school_non_sum.sort_values(by=['month_num'])
steps_pre_closure_wkday_school_non_sum_col = steps_pre_closure_wkday_school_non_sum[['Month_Year_Name', 'Monthly_Avg']]
steps_pre_closure_wkday_school_non_sum_col_dup = steps_pre_closure_wkday_school_non_sum_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')

steps_dur_closure_wkday_school_non_sum = df_closure_tod_wkday_school_non_sum_dup[df_closure_tod_wkday_school_non_sum_dup['closure_status'] == 'During Closure']
steps_dur_closure_wkday_school_non_sum['Monthly_Avg'] = steps_dur_closure_wkday_school_non_sum.groupby(['Month_Year_Name'])['mean_monthly_steps_school'].transform('mean')
steps_dur_closure_wkday_school_non_sum['month_num'] = steps_dur_closure_wkday_school_non_sum['Start_Time_Offset'].dt.month
steps_dur_closure_wkday_school_non_sum = steps_dur_closure_wkday_school_non_sum.sort_values(by=['month_num'])
steps_dur_closure_wkday_school_non_sum_col = steps_dur_closure_wkday_school_non_sum[['Month_Year_Name', 'Monthly_Avg']]
steps_dur_closure_wkday_school_non_sum_col_dup = steps_dur_closure_wkday_school_non_sum_col.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')

In [ ]:
from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(steps_dur_closure_wkday_school_non_sum_col_dup['Monthly_Avg'], steps_pre_closure_wkday_school_non_sum_col_dup['Monthly_Avg'])
print(U1)
print(p*100)

print(p)

In [ ]:
# Summer months weekday steps analysis (pre-, during-, and post-closure)

# Starting dataset used: df_closure_tod_wkday_school
df_closure_tod_wkday_school_only_sum = df_closure_tod_wkday_school[
    (df_closure_tod_wkday_school['Mon_Year'] == 'Jun-2020') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Jul-2020') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Aug-2020') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Jun-2019') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Jul-2019') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Aug-2019') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Jun-2021') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Jul-2021') |
    (df_closure_tod_wkday_school['Mon_Year'] == 'Aug-2021')
]

df_closure_tod_wkday_school_only_sum['Daily_Sum_TOD'] = df_closure_tod_wkday_school_only_sum.groupby(['Start_Time_Date_Offset_x'])['Steps'].transform('sum')
df_closure_tod_wkday_school_only_sum['mean_monthly_steps_school_sum'] = df_closure_tod_wkday_school_only_sum.groupby(['User_UUID'])['Daily_Sum_TOD'].transform('mean')

df_closure_tod_wkday_school_only_sum_dup = df_closure_tod_wkday_school_only_sum.drop_duplicates(subset=['User_UUID', 'mean_monthly_steps_school_sum'], keep='first')

In [ ]:
# Min
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['mean_monthly_steps_school_sum'].min()

# Max
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['mean_monthly_steps_school_sum'].max()

# Mean
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['mean_monthly_steps_school_sum'].mean()

# Std
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['mean_monthly_steps_school_sum'].std()

# Users
df_closure_tod_wkday_school_only_sum_dup.groupby('closure_status')['User_UUID'].nunique()

In [ ]:
df_closure_tod_wkday_school_only_sum_dup_fig = df_closure_tod_wkday_school_only_sum_dup[df_closure_tod_wkday_school_only_sum_dup['closure_status'] != 'Post Closure']

plt.figure(figsize=(100, 40))
sns.set_style("white")
palette = {"Pre Closure": "yellowgreen",
           "During Closure": "lightsalmon"
#            "#2020": "salmon"
          }

ax = sns.boxplot(x="month_only", y="mean_monthly_steps_school_sum", palette=palette, hue='closure_status',
                 data=fig_3_df_closure_tod, linewidth=14, showfliers=False,
                 showmeans=False, width=0.5)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_xlabel('Month', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_ylabel('Number of Steps', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_title('Average Daily Step Count Values for Participants Before and After Lockdown During School Times',
             fontdict={'fontsize': 90, 'fontweight': 'bold'}, y=1.05)

plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0, frameon=True, edgecolor='black',
           title="Pandemic Status", framealpha=0.5)


In [ ]:
df_closure_tod_wkday_school_only_sum_dup_fig = df_closure_tod_wkday_school_only_sum_dup[
    df_closure_tod_wkday_school_only_sum_dup['closure_status'] != 'Post Closure']

plt.figure(figsize=(100, 40))
sns.set_style("white")
palette = {
    "Pre Closure": "yellowgreen",
    "During Closure": "lightsalmon"
#     "#2020": "salmon"
}

ax = sns.boxplot(x="month_only", y="mean_monthly_steps_school_sum", palette=palette, hue="closure_status",
                 data=fig_3_df_closure_tod, linewidth=14, showfliers=False,
                 showmeans=False, width=0.5)

ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_xlabel('Month', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_ylabel('Number of Steps', fontdict={'fontsize': 90, 'fontweight': 'bold'})
ax.set_title('Average Daily Step Count Values for Participants Before and After Lockdown During School Times',
             fontdict={'fontsize': 90, 'fontweight': 'bold'}, y=1.05)

plt.legend(bbox_to_anchor=(1.04, 0.5), loc="center left", borderaxespad=0, frameon=True, edgecolor='black',
           title="Pandemic Status", framealpha=0.5)

In [ ]:
# New one value per month
df_closure_tod_wkday_school_only_sum_dup_month = df_closure_tod_wkday_school_only_sum_dup
df_closure_tod_wkday_school_only_sum_dup_month['mean_monthly_steps_school_sum_per_month'] = df_closure_tod_wkday_school_only_sum_dup_month.groupby(['Month_Year_Name'])['mean_monthly_steps_school_sum'].transform('mean')

df_closure_tod_wkday_school_only_sum_dup_month_dup = df_closure_tod_wkday_school_only_sum_dup_month.drop_duplicates(subset=['mean_monthly_steps_school_sum_per_month', 'Month_Year_Name'], keep='first')

# Min
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_sum_per_month'].min()

# Max
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_sum_per_month'].max()

# Mean
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_sum_per_month'].mean()

# Std
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['mean_monthly_steps_school_sum_per_month'].std()


# Users
df_closure_tod_wkday_school_only_sum_dup_month_dup.groupby('closure_status')['User_UUID'].nunique()

In [ ]:
# Statistical test
steps_pre_closure_wkday_school_only_sum_dup = df_closure_tod_wkday_school_only_sum_dup[df_closure_tod_wkday_school_only_sum_dup['closure_status'] == 'Pre Closure']
steps_pre_closure_wkday_school_only_sum_dup['Monthly_Avg'] = steps_pre_closure_wkday_school_only_sum_dup.groupby(['Month_Year_Name'])['mean_monthly_steps_school_sum'].transform('mean')
steps_pre_closure_wkday_school_only_sum_dup['month_num'] = steps_pre_closure_wkday_school_only_sum_dup['Start_Time_Offset'].dt.month
steps_pre_closure_wkday_school_only_sum_dup = steps_pre_closure_wkday_school_only_sum_dup.sort_values(by=['month_num'])
steps_pre_closure_wkday_school_only_sum_col_dup = steps_pre_closure_wkday_school_only_sum_dup[['Month_Year_Name', 'Monthly_Avg']]
steps_pre_closure_wkday_school_only_sum_col_dup = steps_pre_closure_wkday_school_only_sum_col_dup.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')

steps_dur_closure_wkday_school_only_sum_dup = df_closure_tod_wkday_school_only_sum_dup[df_closure_tod_wkday_school_only_sum_dup['closure_status'] == 'During Closure']
steps_dur_closure_wkday_school_only_sum_dup['Monthly_Avg'] = steps_dur_closure_wkday_school_only_sum_dup.groupby(['Month_Year_Name'])['mean_monthly_steps_school_sum'].transform('mean')
steps_dur_closure_wkday_school_only_sum_dup['month_num'] = steps_dur_closure_wkday_school_only_sum_dup['Start_Time_Offset'].dt.month
steps_dur_closure_wkday_school_only_sum_dup = steps_dur_closure_wkday_school_only_sum_dup.sort_values(by=['month_num'])
steps_dur_closure_wkday_school_only_sum_col_dup = steps_dur_closure_wkday_school_only_sum_dup[['Month_Year_Name', 'Monthly_Avg']]
steps_dur_closure_wkday_school_only_sum_col_dup = steps_dur_closure_wkday_school_only_sum_col_dup.drop_duplicates(subset=['Monthly_Avg', 'Month_Year_Name'], keep='first')
steps_dur_closure_wkday_school_only_sum_col_dup

In [ ]:
from scipy.stats import mannwhitneyu

U1, p = mannwhitneyu(steps_dur_closure_wkday_school_only_sum_col_dup['Monthly_Avg'],
                     steps_pre_closure_wkday_school_only_sum_col_dup['Monthly_Avg'])
print(U1)
print(p*100)
print(p)

In [ ]:


# Sleep
df_sleep = df[df['Type'] == 'sleepDuration']
len(df_sleep)

df_sleep['User UUID'].nunique()

steps_user_id_list = df_valid_days_daycount['User UUID'].nunique()

df_sleep_step_ids = df_sleep[df_sleep['User UUID'].isin(steps_user_id_list)]
df_sleep_step_ids

df_sleep_step_ids['User UUID'].nunique()

df_valid_days_daycount['User UUID'].nunique()

df_valid_days_daycount['User UUID']

In [ ]:
df_sleep_step_ids['Start Time'] = df_sleep_step_ids['Start Time'].apply(pd.to_datetime)
df_sleep_step_ids['End Time'] = df_sleep_step_ids['End Time'].apply(pd.to_datetime)

df_sleep_step_ids['Start Time Offset'] = df_sleep_step_ids['Start Time'] + pd.to_timedelta(df_sleep_step_ids['Time Zone Offset'], unit='m')
df_sleep_step_ids['End Time Offset'] = df_sleep_step_ids['End Time'] + pd.to_timedelta(df_sleep_step_ids['Time Zone Offset'], unit='m')

df_sleep_step_ids['Start Time Date Offset'] = df_sleep_step_ids['Start Time Offset'].dt.date
df_sleep_step_ids['End Time Date Offset'] = df_sleep_step_ids['End Time Offset'].dt.date
df_sleep_step_ids['month_year'] = df_sleep_step_ids['Start Time Offset'].dt.to_period('M')

df_sleep_step_ids['Mon Year'] = df_sleep_step_ids['Start Time Offset'].dt.strftime('%b-%Y')
df_sleep_step_ids['Month Year Name'] = df_sleep_step_ids['Mon Year']

In [ ]:
df_sleep_copy = df_sleep_step_ids
df_sleep_copy_dup = df_sleep_copy.drop_duplicates(subset=['User UUID', 'Start Time', 'End Time', 'Value'], keep='first')
df_sleep_copy_dup

df_sleep_copy_dup['User UUID'].nunique()

In [ ]:
df_sleep_copy_dup['year_only'] = df_sleep_copy_dup['Start Time Offset'].dt.year
df_sleep_copy_dup['month_only'] = df_sleep_copy_dup['Start Time Offset'].dt.strftime("%B")
df_sleep_copy_dup['Start Time Only Time'] = df_sleep_copy_dup['Start Time Offset'].dt.strftime("%H:%M:%S")

# Convert to datetime format to calculate bedtime date
df_sleep_copy_dup['Start Time Only Time Datetime'] = df_sleep_copy_dup['Start Time Only Time'].apply(pd.to_datetime)

# Get only nighttime values
# diff=445416-410584
# change based on today's date
df_sleep_bedtime_calc_allnight = df_sleep_copy_dup[(df_sleep_copy_dup['Start Time Only Time Datetime'] > dt.datetime(2023, 6, 27, 18, 0, 0)) &
                                                   (df_sleep_copy_dup['Start Time Only Time Datetime'] < dt.datetime(2023, 6, 27, 8, 0, 0))]

df_sleep_bedtime_calc_allnight['Start Time Only Time Datetime'].dt.date.unique()

In [ ]:
# Why twice?
df_sleep_bedtime_calc_allnight['Start Time Only Time Datetime'] = df_sleep_bedtime_calc_allnight['Start Time Only Time'].apply(pd.to_datetime)

from datetime import timedelta

df_sleep_bedtime_calc_allnight_val = df_sleep_bedtime_calc_allnight
# change according to today's date
df_sleep_bedtime_calc_allnight_val.loc[(df_sleep_bedtime_calc_allnight_val['Start Time Only Time Datetime'] 
                                        > dt.datetime(2023, 6, 27, 18, 0, 0)), 'Bedtime_Date'] = 
df_sleep_bedtime_calc_allnight_val['Start Time_Date_Offset']
df_sleep_bedtime_calc_allnight_val.loc[(df_sleep_bedtime_calc_allnight_val['Start Time Only Time Datetime'] 
                                        < dt.datetime(2023, 6, 27, 18, 0, 0)), 'Bedtime_Date'] = 
df_sleep_bedtime_calc_allnight_val['Start_Time_Date_Offset'] - timedelta(days=1)

df_sleep_bedtime_calc_allnight_val['Start Time_Offset_dt'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val['Start Time_Offset'])
df_sleep_bedtime_calc_allnight_val['End Time_Offset_dt'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val['End Time_Offset'])

In [ ]:
df_sleep_bedtime_calc_allnight_val['Start_Time_Offset_dt'] = df_sleep_bedtime_calc_allnight_val['Start_Time_Offset_dt'].astype('int64') // 1e9
df_sleep_bedtime_calc_allnight_val['End_Time_Offset_dt'] = df_sleep_bedtime_calc_allnight_val['End_Time_Offset_dt'].astype('int64') // 1e9

df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso'] = df_sleep_bedtime_calc_allnight_val.groupby(['User UUID', 'Bedtime_Date'])['Start_Time_Offset_dt_iso'].transform('min')
df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso'] = df_sleep_bedtime_calc_allnight_val.groupby(['User UUID', 'Bedtime_Date'])['End_Time_Offset_dt_iso'].transform('max')

df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso'], unit='s')
df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso'], unit='s')

df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted_dt'] = df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted'].apply(pd.to_datetime)
df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted_dt'] = df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted'].apply(pd.to_datetime)

df_sleep_bedtime_calc_allnight_val['Bedtime_Min_iso_int_converted_dt'].strftime("%H:%M:%S")
df_sleep_bedtime_calc_allnight_val['Waketime_Max_iso_int_converted_dt'].strftime("%H:%M:%S")

In [7]:
from cmath import rect, phase
from math import radians, degrees

def meanAngle(deg):
    complexDegree = sum(rect(1, radians(d)) for d in deg) / len(deg)
    argument = phase(complexDegree)
    meanAngle = degrees(argument)
    return meanAngle

def meanTime(times):
    t = (time.split(':') for time in times)
    seconds = sum((float(s) + int(m) * 60 + int(h) * 3600) for h, m, s in t)
    
    day = 24 * 60 * 60
    toAngles = [s * 360. / day for s in seconds]
    meanAsAngle = meanAngle(toAngles)
    meanSeconds = meanAsAngle * day / 360.
    
    if meanSeconds < 0:
        meanSeconds += day
    h, m = divmod(meanSeconds, 3600)
    m, s = divmod(m, 60)
    
    return '%02i:%02i:%02i' % (h, m, s)

In [ ]:
df_sleep_bedtime_calc_allnight_val_copy = df_sleep_bedtime_calc_allnight_val

# get month from bedtime
df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date_month_year'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date']).dt.to_period('M')
df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date_month_year_abb'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date']).dt.strftime('%b')

df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date_month_year']

# Do we need this?
df_sleep_bedtime_calc_allnight_val_copy['Mon_Year_Bedtime'] = pd.to_datetime(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date']).dt.strftime('%b-%Y')
df_sleep_bedtime_calc_allnight_val_copy['Month_Year_Name_Bedtime'] = df_sleep_bedtime_calc_allnight_val_copy['Mon_Year_Bedtime']

df_sleep_bedtime_calc_allnight_val_copy['Month_Year_Name_Bedtime'] = df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'].dt.date
df_sleep_bedtime_calc_allnight_val_copy.loc[(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'].dt.date < dt.date(2020, 3, 15)) &
                                            (df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'].dt.date >= dt.date(2020, 3, 1)),
                                            'Month_Year_Name_Bedtime'] = '1-14 Mar-2020'
df_sleep_bedtime_calc_allnight_val_copy.loc[(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'].dt.date < dt.date(2020, 4, 1)) &
                                            (df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'].dt.date >= dt.date(2020, 3, 15)),
                                            'Month_Year_Name_Bedtime'] = '15-31 Mar-2020'
df_sleep_bedtime_calc_allnight_val_copy.loc[(df_sleep_bedtime_calc_allnight_val_copy['Bedtime_Date'].dt.date < dt.date(2020, 4, 1)),
                                            'Month_Year_Name_Bedtime'] = df_sleep_bedtime_calc_allnight_val_copy.Month_Year_Name_Bedtime

In [ ]:
column_names_bt = df_sleep_bedtime_calc_allnight_val_copy.columns.values
df_sleep_bedtime_calc_allnight_val_mean = pd.DataFrame(columns=column_names_bt)

unique_id_avg = df_sleep_bedtime_calc_allnight_val_copy['User UUID'].unique()

for i in unique_id_avg:
    df_id_avg = df_sleep_bedtime_calc_allnight_val_copy[df_sleep_bedtime_calc_allnight_val_copy['User UUID'] == i]
    print(i)

    unique_month = df_id_avg['Month_Year_Name_Bedtime'].unique()

    for j in unique_month:
        df_id_date_avg = df_id_avg[df_id_avg['Month_Year_Name_Bedtime'] == j]
        df_id_date_avg['Average Bedtime'] = meanTime(df_id_date_avg['Bedtime_Min_iso_int_converted_time'])

        # Calculate average wake time.
        df_id_date_avg['Average Waketime'] = meanTime(df_id_date_avg['Waketime_Max_iso_int_converted_time'])

        # Append the average date calculations back to the main dataframe.
        df_sleep_bedtime_calc_allnight_val_mean = df_sleep_bedtime_calc_allnight_val_mean.append(df_id_date_avg)

# Restrict the time to a specific range and print the minimum and maximum dates.
df_sleep_bedtime_calc_allnight_val_mean = df_sleep_bedtime_calc_allnight_val_mean[
    (df_sleep_bedtime_calc_allnight_val_mean['Bedtime Date'] > dt.date(2019, 3, 1)) &
    (df_sleep_bedtime_calc_allnight_val_mean['Bedtime Date'] <= dt.date(2021, 6, 30))
]

print(df_sleep_bedtime_calc_allnight_val_mean['Bedtime Date'].min())
print(df_sleep_bedtime_calc_allnight_val_mean['Bedtime Date'].max())

In [ ]:
df_dleep_bedtime_calc_allnight_val_mean.loc[(df_dleep_bedtime_calc_allnight_val_mean['Bedtime_Date']<dt.date(2020,3,15))
                                            &(df_dleep_bedtime_calc_allnight_val_mean['Bedtime_Date']>dt.date(2019,3,15))]


df_sleep_bedtime_calc_allnight_val_mean.loc[
    (df_sleep_bedtime_calc_allnight_val_mean['Bedtime Date'] < dt.date(2020, 3, 15)) & 
    (df_sleep_bedtime_calc_allnight_val_mean['Bedtime Date'] >= dt.date(2019, 3, 1)),
    'closure_status'] = 'Pre Closure'

df_sleep_bedtime_calc_allnight_val_mean.loc[
    (df_sleep_bedtime_calc_allnight_val_mean['Bedtime Date'] <= dt.date(2021, 3, 31)) & 
    (df_sleep_bedtime_calc_allnight_val_mean['Bedtime Date'] >= dt.date(2020, 3, 15)),
    'closure_status'] = 'During Closure'

df_sleep_bedtime_calc_allnight_val_mean.loc[
    (df_sleep_bedtime_calc_allnight_val_mean['Bedtime Date'] > dt.date(2021, 3, 1))]
    ,'closure_status'] = 'Post Closure'

                                                                                                                         
                                                                                                                         


In [ ]:


# Print the minimum bedtime date grouped by 'closure_status'
print(df_sleep_bedtime_calc_allnight_val_mean.groupby('closure_status')['Bedtime Date'].min())


In [ ]:

# Start of new calculation using the same approach as step  avg

df_sleep_bedtime_calc_allnight_val_mean_dup = df_sleep_bedtime_calc_allnight_val_mean.drop_duplicates(subset=['User UUID', 'Month_Year_Name_Bedtime'], keep='first')


column_names_avg = df_sleep_bedtime_calc_allnight_val_mean_dup.columns.values
df_avg_per_month = pd.DataFrame(columns=column_names_avg)

# Extracting unique month-year combinations
month_avg = df_sleep_bedtime_calc_allnight_val_mean_dup['Month_Year_Name_Bedtime'].unique()
j = 0  # 'j' is initialized but not used in the visible code

# Looping through each month for further calculations
for i in month_avg:
    df_month_avg = 
    df_sleep_bedtime_calc_allnight_val_mean_dup[df_sleep_bedtime_calc_allnight_val_mean_dup['Month_Year_Name_Bedtime'] == i]
   
    
    
    
    

In [ ]:
column_names_avg = df_sleep_bedtime_calc_allnight_val_mean_dup.columns.values
df_avg_per_month = pd.DataFrame(columns=column_names_avg)

# Getting unique 'Month_Year_Name_Bedtime' values from the data.
month_avg = df_sleep_bedtime_calc_allnight_val_mean_dup['Month_Year_Name_Bedtime'].unique()

# Loop through each month-year to calculate average bedtime and waketime per period.
for i in month_avg:
    df_month_avg = df_sleep_bedtime_calc_allnight_val_mean_dup[df_sleep_bedtime_calc_allnight_val_mean_dup['Month_Year_Name_Bedtime'] == i]
    df_month_avg['Average_Bedtime_per_period'] = meanTime(df_month_avg['Average Bedtime'])
    df_month_avg['Average_Waketime_per_period'] = meanTime(df_month_avg['Average Waketime'])
    print(i)

    # Appending the monthly averages to the main DataFrame.
    df_avg_per_month = df_avg_per_month.append(df_month_avg)

# Deduplicating the data based on 'Average_Bedtime_per_period' and 'Month_Year_Name_Bedtime'.
df_avg_per_month_bt_dup = df_avg_per_month.drop_duplicates(subset=['Average_Bedtime_per_period', 'Month_Year_Name_Bedtime'], keep='first')

# Deduplicating the data based on 'Average_Waketime_per_period' and 'Month_Year_Name_Bedtime'.
df_avg_per_month_wt_dup = df_avg_per_month.drop_duplicates(subset=['Average_Waketime_per_period', 'Month_Year_Name_Bedtime'], keep='first')


In [ ]:
# Pre-pandemic bedtime and waketime are being filtered from the data.
Pre_pandemic_bt = df_avg_per_month_bt_dup[df_avg_per_month_bt_dup['closure_status'] == 'Pre Closure']
Pre_pandemic_wt = df_avg_per_month_wt_dup[df_avg_per_month_wt_dup['closure_status'] == 'Pre Closure']

# Calculating pre and post-pandemic mean bedtime and waketime.
Post_pandemic_bt = df_avg_per_month_bt_dup[df_avg_per_month_bt_dup['closure_status'] == 'During Closure']
Post_pandemic_wt = df_avg_per_month_wt_dup[df_avg_per_month_wt_dup['closure_status'] == 'During Closure']

meanTime(Pre_pandemic_bt['Average_Bedtime_per_period'])

In [ ]:
meanTime(Pre_pandemic_wt['Average_Waketime_per_period'])

In [ ]:
meanTime(Post_pandemic_wt['Average_Waketime_per_period'])

In [ ]:
meanTime(Post_pandemic_bt['Average_Bedtime_per_period'])

In [ ]:
def meanTime_a(times, type):
    t = [(time.split(':')) for time in times]
    seconds = [((float(s) + int(m) * 60 + int(h) * 3600) for h, m, s in t)]
    day = 24 * 60 * 60
    toAngles = [s * 360.0 / day for s in seconds]

    x = [(ang + 360) if (ang < 120 and type == 'Night') else ang for ang in toAngles]
    mean = (np.mean(x) - 360) if np.mean(x) > 360 else np.mean(x)
    std = (np.std(x) - 360) if np.std(x) > 360 else np.std(x)

    print(x)
    print(mean)
    print(std)

    for val in [mean, std]:
        meanSeconds = val * day / 360.0
        if meanSeconds < 0:
            meanSeconds += day
        h, m = divmod(meanSeconds, 3600)
        m, s = divmod(m, 60)
        print(f'{int(h):02d}:{int(m):02d}:{int(s):02d}')

In [ ]:
# Summarizing pre-pandemic bedtime data
Pre_pandemic_bt_sum = Pre_pandemic_bt[Pre_pandemic_bt['Month_Year_Name_Bedtime'].isin(['Jun-2019', 'Jul-2019', 'Aug-2019'])]
Pre_pandemic_bt_sum = Pre_pandemic_bt_sum[['Average_Bedtime_per_period', 'Month_Year_Name_Bedtime']]

# Calculating mean time for pre-pandemic bedtime
meanTime(Pre_pandemic_bt_sum['Average_Bedtime_per_period'])

# Calculating mean time for pre-pandemic bedtime at night
meanTime_a(Pre_pandemic_bt_sum['Average_Bedtime_per_period'], 'Night')

# Summarizing pre-pandemic waketime data
Pre_pandemic_wt_sum = Pre_pandemic_wt[Pre_pandemic_wt['Month_Year_Name_Bedtime'].isin(['Jun-2019', 'Jul-2019', 'Aug-2019'])]
Pre_pandemic_wt_sum = Pre_pandemic_wt_sum[['Average_Waketime_per_period', 'Month_Year_Name_Bedtime']]

# Calculating mean time for pre-pandemic waketime
meanTime(Pre_pandemic_wt_sum['Average_Waketime_per_period'])

# Calculating mean time for pre-pandemic waketime during the day
meanTime_a(Pre_pandemic_wt_sum['Average_Waketime_per_period'], 'Day')

In [ ]:
# Rounding average bedtime to the nearest 15 minutes
df_avg_per_month_bt_dup = df_sleep_bedtime_calc_allnight_val_mean_dup
df_avg_per_month_bt_dup['Average_Bedtime_Round_fifteenmin'] = pd.to_datetime(
    pd.to_datetime(df_avg_per_month_bt_dup['Average_Bedtime'], format="%H:%M:%S").dt.round('15min')
).dt.strftime("%H:%M:%S")

# Rounding average waketime to the nearest 15 minutes
df_avg_per_month_wt_dup = df_sleep_bedtime_calc_allnight_val_mean_dup
df_avg_per_month_wt_dup['Average_Waketime_Round_fifteenmin'] = pd.to_datetime(
    pd.to_datetime(df_avg_per_month_wt_dup['Average_Waketime'], format="%H:%M:%S").dt.round('15min')
).dt.strftime("%H:%M:%S")

In [ ]:
df_avg_per_month_bt_dup['Average_Bedtime_Round_fifteenmin'].unique()

# Extracting hours and minutes from the rounded bedtime, converting them to strings
df_avg_per_month_bt_dup['bt_hr'] = df_avg_per_month_bt_dup['Average_Bedtime_Round_fifteenmin'].dt.strftime("%H").astype(str)
df_avg_per_month_bt_dup['bt_min'] = df_avg_per_month_bt_dup['Average_Bedtime_Round_fifteenmin'].dt.strftime("%M").astype(str)

# Converting these to a floating-point representation
df_avg_per_month_bt_dup['bt_float'] = df_avg_per_month_bt_dup['bt_hr'] + '.' + df_avg_per_month_bt_dup['bt_min']


# Similar steps are taken for the waketime data
df_avg_per_month_wt_dup['wt_hr'] = df_avg_per_month_wt_dup['Average_Waketime_Round_fifteenmin'].dt.strftime("%H").astype(str)
df_avg_per_month_wt_dup['wt_min'] = df_avg_per_month_wt_dup['Average_Waketime_Round_fifteenmin'].dt.strftime("%M").astype(str)


In [ ]:
df_avg_per_month_wt_dup_order = df_avg_per_month_wt_dup
df_avg_per_month_wt_dup_order['wt_float'] = df_avg_per_month_wt_dup_order['wt_float'].astype(float)
df_avg_per_month_wt_dup_order['flip_wake'] = 0


df_avg_per_month_wt_dup_order.loc[df_avg_per_month_wt_dup_order['wt_float'] < 12, 'flip_wake'] = \
    df_avg_per_month_wt_dup_order['wt_float'] + 12


df_avg_per_month_wt_dup_order_fig2 = df_avg_per_month_wt_dup_order.drop_duplicates(
    subset=['User UUID', 'Average_Waketime_Round_fifteenmin', 'Month_Year_Name_Bedtime'],
    keep='first'
)

len(df_avg_per_month_wt_dup_order_fig2)

In [ ]:
plt.figure(figsize=(95, 35))
sns.set(font_scale=8)
sns.set_style("white")

# Defining the color palette for the plot based on the 'closure_status'.
palette = {
    'Pre Closure': "indianred",
    'During Closure': "gold"
}

# Creating a boxplot that displays the distribution of 'flip_wake' times across months,
# differentiated by 'closure_status'.
ax = sns.boxplot(
    x='month_only', 
    y='flip_wake',
    data=fig_2_sort_wake,
    hue="closure_status",
    palette=palette,
    linewidth=12,
    showfliers=False  # This disables showing outliers in the plot.
)

# Customizing tick labels and adding labels for x and y axes with specific font sizes and weights.
plt.setp(ax.get_xticklabels(), rotation=0)
ax.set_xlabel("", fontdict={'fontsize': 90, 'fontweight': 'bold'}, y=-4.5)
ax.set_ylabel("Average Daily Wake Time", fontdict={'fontsize': 100, 'fontweight': 'bold'})

# Setting the title of the plot with a specific font size and weight.
ax.set_title(
    "Average Daily Wake Times By Month (rounded to nearest 15 mins) for Participants (Night Sleep)",
    fontdict={'fontsize': 90, 'fontweight': 'bold'},
    y=1.03  # Adjusting the vertical position of the title.
)

# Customizing the legend of the plot with specific font sizes for text and title.
plt.legend(
    loc="upper right",
    fontsize=70,
    title="Year",
    framealpha=0.5
)
plt.setp(ax.get_legend().get_texts(), fontsize='80')  # Setting font size for Legend text.
plt.setp(ax.get_legend().get_title(), fontsize='80') 

In [ ]:
df_avg_per_month_bt_dup_order = df_avg_per_month_bt_dup

# Converting the 'bt_float' column to float and initializing 'flip' column.
df_avg_per_month_bt_dup_order['bt_float'] = df_avg_per_month_bt_dup_order['bt_float'].astype(float)
df_avg_per_month_bt_dup_order['flip'] = 0

# The code seems to be adjusting the 'flip' value based on the time of day.
df_avg_per_month_bt_dup_order.loc[df_avg_per_month_bt_dup_order['bt_float'] > 12, 'flip'] = \
    df_avg_per_month_bt_dup_order['bt_float'] - 12
df_avg_per_month_bt_dup_order.loc[df_avg_per_month_bt_dup_order['bt_float'] < 12, 'flip'] = \
    df_avg_per_month_bt_dup_order['bt_float'] + 12

# Dropping duplicates from the DataFrame for a cleaner dataset.
df_avg_per_month_bt_dup_order_fig2 = df_avg_per_month_bt_dup_order.drop_duplicates(
    subset=['User UUID', 'Average_Bedtime_Round_fifteenmin', 'Month_Year_Name_Bedtime'], 
    keep='first'
)

# Displaying the DataFrame or storing it for further analysis or visualization.
df_avg_per_month_bt_dup_order_fig2

In [ ]:
fig_2_sort = df_avg_per_month_bt_dup_order_fig2

# Filtering the DataFrame for entries during 'During Closure' and 'Pre Closure'
fig_2_sort = fig_2_sort[
    (fig_2_sort['closure_status'] == 'During Closure') | 
    (fig_2_sort['closure_status'] == 'Pre Closure')
]

# Creating a 'Bedtime Date Month Closure' column based on the 'Month_Year_Name_Bedtime' column
fig_2_sort['Bedtime_Date_Month_Closure'] = fig_2_sort['Month_Year_Name_Bedtime'].str[0:3]

# Setting specific date ranges for the 'Bedtime_Date_Month_Closure' based on conditions
fig_2_sort.loc[fig_2_sort['Month_Year_Name_Bedtime'] == '1-14 Mar-2020', 'Bedtime_Date_Month_Closure'] = 'Mar'
fig_2_sort.loc[fig_2_sort['Month_Year_Name_Bedtime'] == '15-31 Mar-2020', 'Bedtime_Date_Month_Closure'] = 'Mar'

# Converting 'Bedtime Date Month Closure' to a categorical type with an ordered list of months
fig_2_sort['month_only'] = pd.Categorical(
    fig_2_sort['Bedtime_Date_Month_Closure'],
    categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
    ordered=True
)

In [ ]:
plt.figure(figsize=(95, 35))
sns.set(font_scale=8)
sns.set_style("white")

# Defining the color palette for different closure statuses
palette = {
    'Pre Closure': "steelblue",
    'During Closure': "skyblue",
    'Post Closure': "salmon"
}

# Creating a seaborn boxplot for the 'month_only' and 'flip' columns in the fig_2_sort DataFrame
ax = sns.boxplot(
    x='month_only',
    y='flip',
    data=fig_2_sort,
    hue='closure_status',
    palette=palette,
    linewidth=12,
    showfliers=False  # This option removes the outliers from the plot.
)

# Customization of tick labels, including rotation for better readability
plt.setp(ax.get_xticklabels(), rotation=0)


# Customizing the labels and title of the plot with specified font sizes and weights
ax.set_xlabel("", fontdict={'fontsize': 90, 'fontweight': 'bold'}, y=-4.5)
ax.set_ylabel("Average Daily Bed Time", fontdict={'fontsize': 100, 'fontweight': 'bold'})
ax.set_title(
    "Average Daily Sleep Times By Month (rounded to nearest 15 mins) for Participants",
    fontdict={'fontsize': 90, 'fontweight': 'bold'},
    y=1.03
)

# Setting up the legend of the plot, including its location, font size, and title
plt.legend(
    loc="center left",
    bbox_to_anchor=(1.01, 0.5),
    frameon=True,
    fontsize=80,
    title="Year",
    framealpha=0.5
)
plt.setp(ax.get_legend().get_texts(), fontsize='80')  # Font size for Legend text
plt.setp(ax.get_legend().get_title(), fontsize='80') 

In [ ]:
df_sleep_bedtime_calc_allnight_val_mean_order['flip_wake'] = 0

# Adjust 'flip_wake' based on the 'Average_Waketime_Round_thirtymin_Float' value being greater than 12
df_sleep_bedtime_calc_allnight_val_mean_order.loc[
    (df_sleep_bedtime_calc_allnight_val_mean_order['Average_Waketime_Round_thirtymin_Float'] > 12),
    'flip_wake'
] = df_sleep_bedtime_calc_allnight_val_mean_order['Average_Waketime_Round_thirtymin_Float'] - 12

# Adjust 'flip_wake' when the 'Average_Waketime_Round_thirtymin_Float' value is less than 12
df_sleep_bedtime_calc_allnight_val_mean_order.loc[
    (df_sleep_bedtime_calc_allnight_val_mean_order['Average_Waketime_Round_thirtymin_Float'] < 12),
    'flip_wake'
] = df_sleep_bedtime_calc_allnight_val_mean_order['Average_Waketime_Round_thirtymin_Float'] + 12


df_sleep_bedtime_calc_allnight_val_mean_order[['flip', 'Average_Bedtime_Round_thirtymin_Float']]
df_sleep_bedtime_calc_allnight_val_mean_order[['flip_wake', 'Average_Waketime_Round_thirtymin_Float']]


df_sleep_bedtime_calc_allnight_val_mean_order['flip'].unique()

In [ ]:
df_sleep_bedtime_calc_allnight_val_mean_order = df_sleep_bedtime_calc_allnight_val_mean

# Initializing a new DataFrame for sleep duration calculations
df_sleep_duration = df_sleep_bedtime_calc_allnight_val_mean_order

# Calculating the total sleep per day by summing up values grouped by 'Bedtime Date' and 'User UUID'
df_sleep_duration['Total Sleep Per Day Bedtime'] = df_sleep_duration.groupby(['Bedtime Date', 'User UUID'])['Value'].transform('sum')

# Converting the 'Total Sleep Per Day Bedtime' to a float
df_sleep_duration['Total Sleep Per Day Bedtime float'] = df_sleep_duration['Total Sleep Per Day Bedtime'].astype(float)

# Calculating the average monthly sleep duration by grouping by 'User UUID' and 'Month_Year_Name_Bedtime'
df_sleep_duration['Average Monthly Sleep Duration Bedtime'] = df_sleep_duration.groupby(['User UUID', 'Month_Year_Name_Bedtime'])['Total Sleep Per Day Bedtime float'].transform('mean')


In [ ]:
# Code to get unique 'Month_Year_Name_Bedtime' values from the DataFrame
unique_month_year_bedtime = df_sleep_duration_fig1['Month_Year_Name_Bedtime'].unique()

# Calculating the minimum average monthly sleep duration by 'closure_status'
min_sleep_duration_by_status = df_sleep_duration_fig1.groupby('closure_status')['Average Monthly Sleep Duration Bedtime'].min()

# Calculating the maximum average monthly sleep duration by 'closure_status'
max_sleep_duration_by_status = df_sleep_duration_fig1.groupby('closure_status')['Average Monthly Sleep Duration Bedtime'].max()

# Calculating the mean average monthly sleep duration by 'closure_status'
mean_sleep_duration_by_status = df_sleep_duration_fig1.groupby('closure_status')['Average Monthly Sleep Duration Bedtime'].mean()

# Calculating the standard deviation of the average monthly sleep duration by 'closure_status'
std_sleep_duration_by_status = df_sleep_duration_fig1.groupby('closure_status')['Average Monthly Sleep Duration Bedtime'].std()

# Calculating the number of unique users in each 'closure_status' group
user_count_by_status = df_sleep_duration_fig1.groupby('closure_status')['User UUID'].nunique()

In [ ]:
# Code to count unique users for each 'closure_status'
user_count_by_closure_status = df_sleep_duration_fig1.groupby('closure_status')['User UUID'].nunique()

# Calculating the mean sleep duration per month
df_sleep_duration_month = df_sleep_duration
df_sleep_duration_month['Avg_Sleep_Duration_Per_Month'] = df_sleep_duration_month.groupby(['Month_Year_Name_Bedtime'])['Average Monthly Sleep Duration_Bedtime'].transform('mean')

# Dropping duplicates to get one mean value per month
df_sleep_duration_month = df_sleep_duration_month.drop_duplicates(subset=['Avg_Sleep_Duration_Per_Month', 'Month_Year_Name_Bedtime'], keep='first')
df_sleep_duration_month

# Calculating the minimum average sleep duration per month grouped by 'closure_status'
min_sleep_duration_per_month_by_status = df_sleep_duration_month.groupby('closure_status')['Avg_Sleep_Duration_Per_Month'].min()

In [ ]:
# Calculating the standard deviation of the average sleep duration per month grouped by 'closure_status'
std_sleep_duration_per_month_by_status = df_sleep_duration_month.groupby('closure_status')['Avg_Sleep_Duration_Per_Month'].std()

# Preparing the data for a statistical test related to 'Pre Closure'
sleep_pre_closure = df_sleep_duration_month[df_sleep_duration_month['closure_status'] == "Pre Closure"]
sleep_pre_closure['Bedtime_Date'] = pd.to_datetime(sleep_pre_closure['Bedtime_Date'])
sleep_pre_closure['month_num'] = sleep_pre_closure['Bedtime_Date'].dt.month
sleep_pre_closure = sleep_pre_closure.sort_values(by=['month_num'])

# Dropping duplicates to prepare for statistical analysis
sleep_pre_closure_dup = sleep_pre_closure.drop_duplicates(subset=['Avg_Sleep_Duration_Per_Month', 'Month_Year_Name_Bedtime'], keep='first')
sleep_pre_closure_dup = sleep_pre_closure_dup[['Avg_Sleep_Duration_Per_Month', 'Month_Year_Name_Bedtime']]
sleep_pre_closure_dup

In [ ]:
from scipy.stats import mannwhitneyu

# Performing a Mann-Whitney U Test on sleep duration data
U1, p = mannwhitneyu(
    sleep_dur_closure_dup['Avg_Sleep_Duration_Per_Month'],
    sleep_pre_closure_dup['Avg_Sleep_Duration_Per_Month']
)
print(U1)
print(p)



In [ ]:
# Preparing a sorted DataFrame for visualization
fig_duration_sort = df_sleep_duration_fig1
fig_duration_sort = fig_duration_sort[
    (fig_duration_sort['closure_status'] == 'During Closure') |
    (fig_duration_sort['closure_status'] == 'Pre Closure')
]

# Setting 'Bedtime_Date_Month_Closure' and categorizing 'month_only'
fig_duration_sort['Bedtime_Date_Month_Closure'] = fig_duration_sort['Month_Year_Name_Bedtime'].str[0:3]
fig_duration_sort.loc[(fig_duration_sort['Month_Year_Name_Bedtime'] == '1-14 Mar-2020'), 'Bedtime_Date_Month_Closure'] = 'Mar'
fig_duration_sort.loc[(fig_duration_sort['Month_Year_Name_Bedtime'] == '15-31 Mar-2020'), 'Bedtime_Date_Month_Closure'] = 'Mar'
fig_duration_sort['month_only'] = pd.Categorical(
    fig_duration_sort['Bedtime_Date_Month_Closure'],
    categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
    ordered=True
)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the figure size and style for the seaborn plot
plt.figure(figsize=(100, 40))
sns.set(font_scale=8)
sns.set_style("white")

# Define a color palette for the closure statuses
palette = {
    'Pre Closure': "yellowgreen",
    'During Closure': "pink"
}

# Create a boxplot with seaborn
ax = sns.boxplot(
    x='month_only',
    y='Average_Monthly_Sleep_Duration_Bedtime',
    data=fig_duration_sort,
    hue='closure_status',
    palette=palette,
    linewidth=14,
    showfliers=False
)

# Customize tick labels and set labels for axes and the plot title with specific font sizes and weights
plt.setp(ax.get_xticklabels(), rotation=0)
ax.set_xlabel("", fontdict={'fontsize': 90, 'fontweight': 'bold'}, y=-4.5)
ax.set_ylabel("Average Daily Sleep Duration (Hours)", fontdict={'fontsize': 100, 'fontweight': 'bold'})
ax.set_title(
    "Average Daily Sleep Duration By Month (rounded to nearest 30 mins) for Participants (Night Sleep)",
    fontdict={'fontsize': 90, 'fontweight': 'bold'}, y=1.03
)

# Configure the legend of the plot
plt.legend(
    loc="center left", bbox_to_anchor=(1.01, 0.5),
    frameon=True, fontsize=70,
    title="Year", framealpha=0.5
)
plt.setp(ax.get_legend().get_texts(), fontsize='90')  # Font size for Legend text
plt.setp(ax.get_legend().get_title(), fontsize='90')  # Font size for Legend title